In [166]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from implementations import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [24]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

### Features cleaning

In [102]:
def print_percentages(array):
    for j in range(array.shape[1]):
        count_col = 0
        for i in range(array.shape[0]):
            if array[i,j] == -999:
                count_col += 1
        count_col = (count_col/array.shape[0])*100
        print(f'For the feature {j+1} : {count_col} % of missing values')

In [288]:
# standardize the matrix (not affected by undetermined values)
def standardize(tX, affected = False) :
    tX_standardized = tX.copy()
    for feature in tX_standardized.T : 
        mask = (feature != -999)
        i_mask = (feature == -999)
        if affected :
            feature[i_mask] = feature[mask].mean() # replace undetermined value by 0
            feature = (feature-feature.mean())/feature.std() # standardize
        else :
            feature[i_mask] = 0 # replace undetermined value by mean of column
            feature[mask] = (feature[mask]-feature[mask].mean())/feature[mask].std() # standardize
    tX_standardized.T[22] = tX.T[22]
    return tX_standardized

In [289]:
# standardize the matrix (affected by undetermined values)
tX_standardized = standardize(tX)
tX_standardized_af = standardize(tX, affected = True)

In [224]:
# Create mask removing columns with unvalid data
remove_na_mask = []
for j in range(tX.shape[1]):
    remove_na_mask.append(not (-999 in tX[:,j]))

In [223]:
# Create mask removing all columns with unvalid or 0 data
remove_all_mask = []
for j in range(tX.shape[1]):
    remove_all_mask.append(not (-999 in tX[:,j]))
remove_all_mask[29] = False
remove_all_mask[22] = False

In [290]:
clean_cols = tX_standardized[:, remove_na_mask]

In [214]:
# for i, feature in enumerate(tX.T) :
#    print("Feature " , i+1)
#    plt.plot(feature[:1000], y[:1000], 'kp', alpha = 0.002)
#    plt.show()

In [204]:
initial_w = np.zeros(clean_cols.shape[1])
lambda_ = 50
weights, loss = least_squares(y, clean_cols)
print(weights, loss)

[-8.29807435e-03 -3.33837373e-03 -4.11087771e-04  1.17229552e-01
 -1.72596844e-03 -3.15300979e+00 -3.16695463e-01  1.24537010e-01
  3.15622641e+00 -7.44030239e-04 -1.12894830e-03  3.16597942e+00
  3.09221308e-04  7.21687868e-04  2.06236580e-03  4.95856184e-04
 -1.06989918e-03  1.70005593e-02  3.15338848e+00] 0.36243002340702934


In [207]:
w_0 = np.zeros(30)
w_0[mask] = weights
weights = w_0

## Generate predictions and save ouput in csv format for submission:

In [206]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [208]:
rmse_loss = compute_rmse(y, tX, weights)
mse_loss = compute_mse(y, tX, weights)
print('rmse ', rmse_loss)
print('mse ', mse_loss)
print('loss found ', loss)
np.dot(tX_test, weights)

rmse  0.851387130989222
mse  0.36243002340702934
loss found  0.36243002340702934


array([-0.79564754, -0.65633087, -0.2681402 , ...,  0.01219161,
        0.14272404, -0.64825036])

In [210]:
OUTPUT_PATH = '../data/submission_file.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)